# CVA bump sensitivities

In this notebook, we calculate linear bump sensitivities, smart bump sensitivities, and benchmark bump sensitivities for CVA. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from learning.cva_estimator_portfolio_int_pl import CVAEstimatorPortfolioInt
from simulation.diffusion_engine_pl import DiffusionEngine
import time
import torch
import pickle
import gc
import pandas as pd

In [2]:
from numba import NumbaPerformanceWarning
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=NumbaPerformanceWarning)

In [3]:
np.random.seed(0)
torch.manual_seed(0)
seed_diff = 0
early_at = [0.01]
time_only = False

In [4]:
torch.backends.cudnn.benchmark = False # don't allow cudnn to tune for every input size
torch.backends.cudnn.enabled = True

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Simulation parameters

* `T`: portfolio maturity;
* `num_coarse_steps`: # of coarse time-steps, *ie* steps at which we price and learn;
* `dT`: size of a coarse time-step (uniform time-stepping), should be equal to the simulation horizon in years divided by `num_coarse_steps`;
* `num_fine_per_coarse`: # of fine time-steps per coarse time-step, those are steps through which the numerical diffusions are stepping;
* `dt`: should be equal to `dT/num_fine_per_coarse`;
* `num_paths`: # of diffusion paths ($M$ in \[1\]);
* `num_inner_paths`: # of inner-paths for the Nested Monte Carlo benchmark, set to `1` if no NMC benchmark is needed;
* `num_defs_per_path`: # of default simulations given each diffusion path ($N$ in \[1\]);
* `num_rates`: # of economies, each represented by a 1-factor short-rate;
* `num_spreads`: # of counterparties + 1, with one stochastic spread for each counterparty and one for the bank itself;
* `R`: correlation matrix, should be `2*num_rates-1+num_spreads` by `2*num_rates-1+num_spreads`;

As for the order in which the stochastic diffusion factors are stored, the first `num_rates` components are the short-rates, the next `num_rates-1` are the associated cross-currency exchange rates against the reference currency (which is assumed to be the first, *ie* with id 0), and the last `num_spreads` are the stochastic intensity processes where the first one is for the bank and the rest are for the counterparties.

As for the diffusion parameters, they can be set using the following correspondence with the notation in Appendix B of \[1\]:
* `rates_params['a'][e]` $\leftrightarrow a^{\langle e\rangle}$;
* `rates_params['b'][e]` $\leftrightarrow b^{\langle e\rangle}$;
* `rates_params['sigma'][e]` $\leftrightarrow \sigma^{r, \langle e\rangle}$;
* `fx_params['vol'][e]` $\leftrightarrow \sigma^{\chi, \langle e\rangle}$;
* `spreads_params['a'][c]` $\leftrightarrow \alpha^{\langle c\rangle}$;
* `spreads_params['b'][c]` $\leftrightarrow \delta^{\langle c\rangle}$;
* `spreads_params['vvol'][c]` $\leftrightarrow \nu^{\langle c\rangle}$;

Finally, for the products, one can set them in their `specs` arrays below. We invite the user to see the pricing functions in `compile_cuda_diffuse_and_price` (simulation/kernels.py) for more details on how the product specifications are used.

In the following, we fill the diffusion parameters and the product specs randomly.

**\[1\]** Abbas-Turki, L., S. Crépey, and B. Saadeddine (2023).  Pathwise CVA regressions with oversimulated defaults. Mathematical Finance 33(2), pages 274-307, 2023.

In [6]:
horizon = 10
num_fine_steps = 2500
dt = horizon/num_fine_steps
num_fine_per_coarse = 25 # 8, 16, 32, 64, 128
dT = num_fine_per_coarse*dt
num_coarse_steps = num_fine_steps//num_fine_per_coarse
num_paths = 2**17
num_inner_paths = 1
num_defs_per_path = 1 #256

num_rates = 10
num_spreads = 9
R = np.eye(2*num_rates-1+num_spreads, dtype=np.float32) # we set the correlation matrix to the identity matrix, although not needed
initial_values = np.empty(2*num_rates-1+num_spreads, dtype=np.float32)
initial_defaults = np.empty((num_spreads-1+7)//8, dtype=np.int8)

# rates diffusion parameters
rates_params = np.empty(num_rates, dtype=[('a', '<f4'), ('b', '<f4'), ('sigma', '<f4')])
rates_params['a'] = np.random.normal(0.5, 0.05, num_rates).round(4)
rates_params['b'] = np.random.normal(0.03, 0.003, num_rates).round(4)
rates_params['sigma'] = np.abs(np.random.normal(0.01, 0.001, num_rates)).round(4)
initial_values[:num_rates] = 0.01

# FX diffusion parameters
fx_params = np.empty(num_rates-1, dtype=[('vol', '<f4')])
fx_params['vol'] = np.abs(np.random.normal(0.5, 0.05, num_rates-1)).round(4)
initial_values[num_rates:2*num_rates-1] = 1

# stochastic intensities diffusion parameter7
spreads_params = np.empty(num_spreads, dtype=[('a', '<f4'), ('b', '<f4'), ('vvol', '<f4')])
spreads_params['a'] = np.random.normal(0.7, 0.07, num_spreads)
spreads_params['b'] = np.random.normal(0.04, 0.004, num_spreads)
spreads_params['vvol'] = np.abs(np.random.normal(0.1, 0.01, num_spreads))
initial_values[2*num_rates-1:] = 0.015

# initial default indicators
initial_defaults[:] = 0

# length of simulated path on the GPU (paths are then simulated by chunks of cDtoH_freq until maturity)
cDtoH_freq = 64 # 20

# product specs (DO NOT use the ZCs)
num_vanillas = 0#500 if portfolio == 'call' else 0
vanilla_specs = np.empty(num_vanillas,
                            dtype=[('maturity', '<f4'), ('notional', '<f4'),
                                ('strike', '<f4'), ('cpty', '<i4'),
                                ('undl', '<i4'), ('call_put', '<b1')])
vanilla_specs['maturity'] = np.random.uniform(0.1,9.5,num_vanillas)
vanilla_specs['notional'] = 1000. *     ((np.random.choice((-1, 1), num_vanillas, p=(0.5, 0.5)))
        * np.random.choice(range(1, 6), num_vanillas))
vanilla_specs['strike'] = abs(1. * np.random.uniform(0.9,1.1, num_vanillas))
vanilla_specs['cpty'] = np.random.randint(0, num_spreads-1, num_vanillas, np.int32)  # Counterparty with which the swap was entered into
vanilla_specs['undl'] = np.random.randint(0, num_rates-1, num_vanillas, np.int32)  # Underlying currency
vanilla_specs['call_put'] = np.random.choice((True, False), num_vanillas, p=(0.5, 0.5))

num_irs = 500
irs_specs = np.empty(num_irs,
                     dtype=[('first_reset', '<f4'), ('reset_freq', '<f4'),
                            ('notional', '<f4'), ('swap_rate', '<f4'),
                            ('num_resets', '<i4'), ('cpty', '<i4'),
                            ('undl', '<i4')])

irs_specs['first_reset'] = 0.  # First reset date in the swaps
irs_specs['reset_freq'] = 75*dt # 0.2  # Reset frequency
irs_specs['notional'] = 10000. *  ((np.random.choice((-1, 1), num_irs, p=(0.5, 0.5))) * np.random.choice(range(1, 11), num_irs)).round(4)  # Notional of the swaps
irs_specs['swap_rate'] = np.abs(np.random.normal(0.03, 0.001, num_irs)).round(4)  # Swap rate
irs_specs['num_resets'] = np.random.randint(int((1+dt)/(75*dt)), num_fine_steps//75+1, num_irs, np.int32) # Number of resets (num_resets*reset_freq should be equal to the desired maturity)
irs_specs['cpty'] = np.random.randint(0, num_spreads-1, num_irs, np.int32)  # Counterparty with which the swap was entered into
irs_specs['undl'] = np.random.randint(0, num_rates, num_irs, np.int32)  # Underlying currency

num_zcs = 0
zcs_specs = np.empty(num_zcs, 
                     dtype=[('maturity', '<f4'), ('notional', '<f4'),
                            ('cpty', '<i4'), ('undl', '<i4')])

In [7]:
initial_g_diff_params = np.concatenate([pd.DataFrame(rates_params).to_numpy().T.ravel(),pd.DataFrame(fx_params).to_numpy().T.ravel(),
               np.zeros(num_rates-1), pd.DataFrame(spreads_params).to_numpy().T.ravel()], dtype= np.float32).T

The parameters are divided into ten groups. They are:
1. Initial values of interest rates;
2. Initial values of foreigner exchanges;
3. Initial values of default intensities;
4. Mean-reversion rate of interest rates;
5. Long-term mean of interest rates;
6. Volatility of interest rates;
7. Volatility of foreigner exchanges;
8. Mean-reversion rate of default intensities;
9. Long-term mean of default intensities;
10. Volatility of default intensities.

In [8]:
# Grouping initial values and parameters
num_diff = initial_values.size
num_para = initial_g_diff_params.size + initial_values.size
group = dict()
group[0] = np.arange(num_rates) #np.delete(np.arange(num_rates),[0,2]) # ini rate
group[1] = np.arange(num_rates, 2*num_rates-1) # ini fx
group[2] = np.arange(2*num_rates, 2*num_rates + num_spreads- 1) # ini spread
group[3] = np.arange(num_diff, num_diff + num_rates) # speed rate
group[4] = np.arange(num_diff+ num_rates, num_diff + 2*num_rates) # long mean rate
group[5] = np.arange(num_diff+ 2*num_rates, num_diff + 3*num_rates) # vol rate
group[6] = np.arange(num_diff+ 3*num_rates, num_diff + 4*num_rates-1)  # vol fx
group[7] = np.arange(num_diff+ 5*num_rates-1, num_diff+ 5*num_rates-1+num_spreads-1)  # speed spread
group[8] = np.arange(num_diff+ 5*num_rates-1+num_spreads, num_diff+ 5*num_rates-1+2*num_spreads-1) # long mean spread
group[9] = np.arange(num_diff+ 5*num_rates-1+2*num_spreads, num_diff+ 5*num_rates-1+3*num_spreads-1) # vol spread
index_para = np.concatenate(list(group.values()))
features0 = np.concatenate([initial_values, initial_g_diff_params]).reshape(1,-1)
features0 = features0[:,index_para]

In [9]:
device = torch.device('cuda:1')

num_steps_per_reset = int((irs_specs['reset_freq'][0]+dt)/(dT))
prev_reset_arr = (np.arange(num_coarse_steps+1)-1)//num_steps_per_reset*num_steps_per_reset

In [10]:
with torch.cuda.device(device):
    _evt_start = torch.cuda.Event(enable_timing=True)
    _evt_stop = torch.cuda.Event(enable_timing=True)

The interest rate swaps are not fairly priced in the provided specs. The specs needs to be updated by running the diffusion. 

In [11]:
def update_irs_specs(initialValue, swapSpec):

    diffusion_engine_ = DiffusionEngine(50, 50, num_coarse_steps, dT, num_fine_per_coarse, dt,
                                        num_paths, num_inner_paths, num_defs_per_path, 
                                        num_rates, num_spreads, R, rates_params, fx_params, 
                                        spreads_params, vanilla_specs, swapSpec, zcs_specs,
                                        initialValue, initial_defaults, cDtoH_freq, device.index, 
                                        pathwise_diff_para=None, seed=1, early_pricing_date=early_at)

    diffusion_engine_.generate_batch(fused=True, nested_cva_at=None, indicator_in_cva=False, set_irs_at_par=True)
    irs_specs_ = diffusion_engine_.irs_specs.copy()

    return irs_specs_

In [12]:
irs_specs_ = update_irs_specs(initial_values, irs_specs)

Successfully compiled all kernels.
cuda_diffuse_and_price elapsed time: 7806.508 ms


In [13]:
def bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initialValue, pathwise_diff_params_, seed, return_features=False, num_paths=num_paths):
    # generate CVA labels for given shocks
    time_rein = time.time()
    diffusion_engine_._reinitialize(initialValue, pathwise_diff_params_)
    time_rein = time.time() - time_rein
    time_diffusion = time.time()
    diffusion_engine_.reset_rng_states(seed)
    diffusion_engine_.generate_batch(fused=True, nested_cva_at=None, indicator_in_cva=False, set_irs_at_par=False)
    
    cva_portfolio_label_gen_ = cva_estimator_portfolio_._build_labels()
    timesteps = range(num_coarse_steps, 0, -1)
    label_ = np.empty((num_coarse_steps+1, num_defs_per_path*num_paths), dtype=np.float32)
    _v__ = label_.reshape((num_coarse_steps+1, num_paths* num_defs_per_path,1))

    for t in timesteps:
        _v__[t] = next(cva_portfolio_label_gen_)
    _v__[0] = next(cva_portfolio_label_gen_)
    
    label0 = label_[0:1].mean(axis = 1)
    time_diffusion = time.time() - time_diffusion
    
    MtM0 = diffusion_engine_.mtm_by_cpty.sum(axis = 1)[0:2].mean(axis = 1)

    d_ = {}
    d_['CVA'] = label0[0]
    d_['labels'] = label_[0:1]
    if return_features:
        d_['features'] = diffusion_engine_.pathwise_diff_para.T
    return d_, time_rein, time_diffusion

# Linear bump sensitivities

In [14]:
def pathwise_diff_params_group( seed = 0, l_g = None, return_true_cov=False):
    # generate gaussian shocks with modulations for groups
    randomized_std = dict()
    randomized_std[0], randomized_std[1], randomized_std[2] = 0.02, 0.02, 0.02
    randomized_std[3], randomized_std[4], randomized_std[5], randomized_std[6] = 0.02, 0.02, 0.04, 0.04
    randomized_std[7], randomized_std[8], randomized_std[9] = 0.02, 0.02, 0.04
    noise = np.zeros((num_para, num_paths), dtype = np.float32)
    if l_g is None:
        l_g = np.arange(10)
    
    std_rel = np.ones(num_para)
    batch_paths = np.zeros(10)
    bs = (num_paths)//len(l_g)
    for i in range(len(l_g)):
        std_rel[group[l_g[i]]] *= randomized_std[l_g[i]]
        eff_i = len(group[l_g[i]])
        if i<len(l_g) - 1:
            bs_small = bs
            batch_paths[l_g[i]] = bs
            noise[group[l_g[i]], (i)*bs:(i+1)*bs] = np.array(np.random.RandomState(seed+i).multivariate_normal(np.zeros(eff_i), np.eye(eff_i), size=(bs_small)).T, dtype=np.float32).round(4)
            noise[group[l_g[i]], (i)*bs:(i+1)*bs] *= randomized_std[l_g[i]]
        else:
            eff_path = num_paths-bs*i
            batch_paths[l_g[i]] = eff_path
            
            noise[group[l_g[i]], i*bs:i*bs+eff_path] = np.array(np.random.RandomState(seed+i).multivariate_normal(np.zeros(eff_i), np.eye(eff_i),size = ( eff_path)).T, dtype = np.float32).round(4)
            noise[group[l_g[i]], i*bs:i*bs+eff_path] *= randomized_std[l_g[i]]
    
    if return_true_cov:
        return noise, std_rel, np.array(batch_paths)
    return noise

In [15]:
def linear_sensis(training_data):
    # calculating sensitivities by linear regression
    features_all = training_data['features']
    labels_all = training_data['labels'].T
    labels_bar_all = training_data['labels_bar'].T
    count = np.zeros(num_para)
    for i in group.keys():
        count[group[i]] += training_data['batch_paths'][i]
    features_ref = training_data['features_ref']
    cov_theory = training_data['std_rel']**2 * features_ref ** 2
    centered_features = features_all - features_ref
    cov_coef_linear = ((labels_all - labels_bar_all).T @ centered_features)
    cov_coef_std = np.std((labels_all - labels_bar_all) * centered_features, axis=0)
    rel_se = cov_coef_std / cov_coef_linear * np.sqrt(centered_features.shape[0])
    return cov_coef_linear[0, :] / (cov_theory * count) / 2, rel_se[0, :]

In [16]:
overhead_linear = time.time()
diffusion_engine_ = DiffusionEngine(50, 50, num_coarse_steps, dT, num_fine_per_coarse, dt,
                                        num_paths, num_inner_paths, num_defs_per_path, 
                                        num_rates, num_spreads, R, rates_params, fx_params, 
                                        spreads_params, vanilla_specs, irs_specs_, zcs_specs,
                                        initial_values, initial_defaults, cDtoH_freq, device.index, 
                                        pathwise_diff_para = None, seed = 0, early_pricing_date =early_at)

        
cva_estimator_portfolio_ = CVAEstimatorPortfolioInt(prev_reset_arr, True, False, False, 0, diffusion_engine_, 
                                            device, 1, 2*(num_rates+num_spreads), 
                                            (num_paths*num_defs_per_path)//8, 
                                            16, 0.01, 0, reset_weights=False, linear=False, 
                                            best_sol=True, refine_last_layer=True)
overhead_full = time.time() - overhead_linear

Successfully compiled all kernels.


In [17]:
linear_data = {}
time_rand_linear = time.time()
pathwise_diff_params, std_rel, batch_paths = pathwise_diff_params_group(seed = 1000, return_true_cov=True)
time_rand_linear = time.time() - time_rand_linear
training, time_rein_linear, time_simul_linear = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, pathwise_diff_params, seed=seed_diff, return_features=True)
training_bar, time_rein_linear_bar, time_simul_linear_bar = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, -pathwise_diff_params, seed=seed_diff, return_features=True)
linear_data['std_rel'] = std_rel
linear_data['batch_paths'] = batch_paths
linear_data['features'] = training['features']
linear_data['labels'] = training['labels']
linear_data['features_bar'] = training_bar['features']
linear_data['labels_bar'] = training_bar['labels']
linear_data['features_ref'] = np.concatenate((initial_values, initial_g_diff_params))
linear_data['group'] = group


cuda_diffuse_and_price elapsed time: 7512.413 ms
cuda_diffuse_and_price elapsed time: 7783.829 ms


In [18]:
time_regression = time.time()
linear_delta, rel_delta_se = linear_sensis(linear_data)
time_regression = time.time() - time_regression
linear_delta_se = rel_delta_se * linear_delta

/tmp/ipykernel_3680973/3769038574.py:14: RuntimeWarning: invalid value encountered in divide
  rel_se = cov_coef_std / cov_coef_linear * np.sqrt(centered_features.shape[0])
/tmp/ipykernel_3680973/3769038574.py:15: RuntimeWarning: invalid value encountered in divide
  return cov_coef_linear[0, :] / (cov_theory * count) / 2, rel_se[0, :]


In [19]:
linear_data['delta'] = linear_delta
linear_data['delta_sd'] = linear_delta_se
linear_timing = {'overhead':overhead_full, 'rand':time_rand_linear, 'rein':(time_rein_linear, time_rein_linear_bar), 'simul':(time_simul_linear, time_simul_linear_bar), 'regression':time_regression}
linear_data['timing'] = linear_timing

# Smart bump sensitivities

In [20]:
overhead_smart = time.time()
diffusion_engine_ = DiffusionEngine(50, 50, num_coarse_steps, dT, num_fine_per_coarse, dt,
                                        num_paths, num_inner_paths, num_defs_per_path, 
                                        num_rates, num_spreads, R, rates_params, fx_params, 
                                        spreads_params, vanilla_specs, irs_specs_, zcs_specs,
                                        initial_values, initial_defaults, cDtoH_freq, device.index, 
                                        pathwise_diff_para = None, seed = 0, early_pricing_date =early_at)

        
cva_estimator_portfolio_ = CVAEstimatorPortfolioInt(prev_reset_arr, True, False, False, 0, diffusion_engine_, 
                                            device, 1, 2*(num_rates+num_spreads), 
                                            (num_paths*num_defs_per_path)//8, 
                                            16, 0.01, 0, reset_weights=False, linear=False, 
                                            best_sol=True, refine_last_layer=True)
overhead_smart = time.time() - overhead_smart

Successfully compiled all kernels.


In [21]:
def diff_rapid_bump(shock=0.01):
    # generating shock for smart bump sensitivities
    l_g = index_para
    noise = np.zeros((num_para, num_paths), dtype = np.float32)
    bs = (num_paths)//len(l_g)
    for i in range(len(l_g)):
        if i<len(l_g) - 1:
            noise[l_g[i], (i)*bs:(i+1)*bs] += shock
        else:
            eff_path = num_paths-bs*i
            noise[l_g[i], i*bs:i*bs+eff_path] += shock
    return noise

In [22]:
time_rand_smart = time.time()
rapid_bump_diff_para = diff_rapid_bump()
time_rand_smart = time.time() - time_rand_smart
d_smart, time_rein_smart, time_simul_smart = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, rapid_bump_diff_para, seed=seed_diff, return_features=True)
d_smart_bar, time_rein_smart_bar, time_simul_smart_bar = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, -rapid_bump_diff_para, seed=seed_diff, return_features=True)

cuda_diffuse_and_price elapsed time: 7819.677 ms
cuda_diffuse_and_price elapsed time: 7827.761 ms


In [23]:
labels_smart = d_smart['labels'].reshape(-1,1)
labels_smart_bar = d_smart_bar['labels'].reshape(-1,1)
smart_delta = np.zeros_like(linear_delta)
smart_delta_se = np.zeros_like(linear_delta)
bs = (num_paths)//len(index_para)

time_sub_smart = time.time()
for i in range(len(index_para)):
    j = len(index_para)-1 -i
    if i!= len(index_para)-1:
        l_smart = labels_smart[i*bs:(i+1)*bs]
        l_smart_bar = labels_smart_bar[i*bs:(i+1)*bs]
    elif i==len(index_para)-1:
        l_smart =  labels_smart[i*bs:]
        l_smart_bar =  labels_smart_bar[i*bs:]
    smart_delta[index_para[i]] = ((l_smart - l_smart_bar)/2/0.01/features0[0,i]).mean()
    smart_delta_se[index_para[i]] = ((l_smart - l_smart_bar)/2/0.01/features0[0,i]).std()/ np.sqrt(len(l_smart_bar))
time_sub_smart = time.time() - time_sub_smart

In [24]:
smart_data = {}
smart_data['delta'] = smart_delta
smart_data['delta_sd'] = smart_delta_se
smart_timing = {'overhead':overhead_smart, 'rand':time_rand_smart, 'rein':(time_rein_smart, time_rein_smart_bar), 'simul':(time_simul_smart, time_simul_smart_bar), 'sub':time_sub_smart}
smart_data['timing'] = smart_timing

# Benchmark bump sensitivities

Both $\Delta$ and $\Gamma$ are calculated using benchmark bump method.

In [25]:
def pathwise_diff_params_verify(chosen_paras, shock=0.01):
    # generating bump for a certain parameter
    a = np.zeros((num_para, num_paths), dtype = np.float32)
    a[chosen_paras,:] = np.ones((num_paths,), dtype = np.float32)*shock
    return a

In [26]:
bench_delta = np.zeros(num_para)
bench_gamma = np.zeros(num_para)
bench_delta_se = np.zeros_like(bench_delta)
time_plus_rein = np.zeros_like(bench_delta)
time_minus_rein = np.zeros_like(bench_delta)
time_plus_simul = np.zeros_like(bench_delta)
time_minus_simul = np.zeros_like(bench_delta)
time_bench_sub = []
time_bench_overhead = []
time_bench_simul = []
time_bench_rand = []
bench_count = 0

In [27]:
overhead_bench = time.time()
diffusion_engine_ = DiffusionEngine(50, 50, num_coarse_steps, dT, num_fine_per_coarse, dt,
                                        num_paths, num_inner_paths, num_defs_per_path, 
                                        num_rates, num_spreads, R, rates_params, fx_params, 
                                        spreads_params, vanilla_specs, irs_specs_, zcs_specs,
                                        initial_values, initial_defaults, cDtoH_freq, device.index, 
                                        pathwise_diff_para = None, seed = 0, early_pricing_date =early_at)

        
cva_estimator_portfolio_ = CVAEstimatorPortfolioInt(prev_reset_arr, True, False, False, 0, diffusion_engine_, 
                                            device, 1, 2*(num_rates+num_spreads), 
                                            (num_paths*num_defs_per_path)//8, 
                                            16, 0.01, 0, reset_weights=False, linear=False, 
                                            best_sol=True, refine_last_layer=True)
overhead_bench = time.time() - overhead_bench
time_bench_overhead.append(overhead_bench)

Successfully compiled all kernels.


In [28]:
baseline_result, time_baseline_rein, time_baseline_simul = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, None, seed=seed_diff, return_features=False)


cuda_diffuse_and_price elapsed time: 7799.952 ms


In [29]:
for k in range(initial_g_diff_params.size-1,-1,-1):    
    bump = 0.01 * initial_g_diff_params[k]
    if bump < 1e-7:
        continue
    
    time_rand_bench = time.time()
    pathwise_diff_params = pathwise_diff_params_verify(initial_values.size+k,0.01)
    time_rand_bench = time.time() - time_rand_bench
    bumpedValuePlus, time_plus_rein[initial_values.size+k], time_plus_simul[initial_values.size+k] = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, pathwise_diff_params, seed=seed_diff)
    
    time_rand_bench_bar = time.time()
    pathwise_diff_params = pathwise_diff_params_verify(initial_values.size+k,-0.01)
    time_rand_bench_bar = time.time() - time_rand_bench_bar
    bumpedValueMinus, time_minus_rein[initial_values.size+k], time_minus_simul[initial_values.size+k] = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, pathwise_diff_params, seed=seed_diff)
    
    time_bench_simul.append(time_minus_simul[initial_values.size+k] + time_plus_simul[initial_values.size+k])
    time_bench_rand.append(time_rand_bench + time_rand_bench_bar)

    time_sub_bench = time.time()
    bench_delta[initial_values.size+k] = ((bumpedValuePlus['labels'][0, :] - bumpedValueMinus['labels'][0, :]) / 2 / bump).mean()
    bench_delta_se[initial_values.size+k] = ((bumpedValuePlus['labels'][0, :] - bumpedValueMinus['labels'][0, :]) / 2 / bump).std() / np.sqrt(num_paths)
    time_sub_bench = time.time() - time_sub_bench
    time_bench_sub.append(time_sub_bench)
    
    bench_gamma[initial_values.size+k] = ((bumpedValuePlus['labels'][0, :] + bumpedValueMinus['labels'][0, :] - 2 * baseline_result['labels'][0, :])).mean() /  np.square(bump)

cuda_diffuse_and_price elapsed time: 7809.076 ms
cuda_diffuse_and_price elapsed time: 7826.345 ms
cuda_diffuse_and_price elapsed time: 7778.158 ms
cuda_diffuse_and_price elapsed time: 7855.384 ms
cuda_diffuse_and_price elapsed time: 7838.65 ms
cuda_diffuse_and_price elapsed time: 7846.273 ms
cuda_diffuse_and_price elapsed time: 7848.303 ms
cuda_diffuse_and_price elapsed time: 4057.254 ms
cuda_diffuse_and_price elapsed time: 7928.501 ms
cuda_diffuse_and_price elapsed time: 7954.588 ms
cuda_diffuse_and_price elapsed time: 7955.731 ms
cuda_diffuse_and_price elapsed time: 7955.213 ms
cuda_diffuse_and_price elapsed time: 7873.903 ms
cuda_diffuse_and_price elapsed time: 7921.374 ms
cuda_diffuse_and_price elapsed time: 7871.728 ms
cuda_diffuse_and_price elapsed time: 7775.598 ms
cuda_diffuse_and_price elapsed time: 7826.003 ms
cuda_diffuse_and_price elapsed time: 7825.372 ms
cuda_diffuse_and_price elapsed time: 7861.194 ms
cuda_diffuse_and_price elapsed time: 7808.357 ms
cuda_diffuse_and_pric

In [30]:
for k in range(initial_values.size):
    bump = 0.01 * initial_values[k]
    if bump < 1e-7:
        continue
    
    time_rand_bench = time.time()
    pathwise_diff_params = pathwise_diff_params_verify(k,0.01)
    time_rand_bench = time.time() - time_rand_bench
    bumpedValuePlus, time_plus_rein[k], time_plus_simul[k] = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, pathwise_diff_params, seed=seed_diff)

    time_rand_bench_bar = time.time()
    pathwise_diff_params = pathwise_diff_params_verify(k,-0.01)
    time_rand_bench_bar = time.time() - time_rand_bench_bar
    bumpedValueMinus, time_minus_rein[k], time_minus_simul[k] = bump_CVA(diffusion_engine_, cva_estimator_portfolio_, initial_values, pathwise_diff_params, seed=seed_diff)
    
    time_bench_simul.append(time_minus_simul[k] + time_plus_simul[k])
    time_bench_rand.append(time_rand_bench + time_rand_bench_bar)

    time_sub_bench = time.time()
    bench_delta[k] = ((bumpedValuePlus['labels'][0, :] - bumpedValueMinus['labels'][0, :]) / 2 / bump).mean()
    bench_delta_se[k] = ((bumpedValuePlus['labels'][0, :] - bumpedValueMinus['labels'][0, :]) / 2 / bump).std() / np.sqrt(num_paths)
    time_sub_bench = time.time() - time_sub_bench
    time_bench_sub.append(time_sub_bench)

    bench_gamma[k] = ((bumpedValuePlus['labels'][0, :] + bumpedValueMinus['labels'][0, :] - 2 * baseline_result['labels'][0, :])).mean() /  np.square(bump)

cuda_diffuse_and_price elapsed time: 7872.289 ms
cuda_diffuse_and_price elapsed time: 7841.332 ms
cuda_diffuse_and_price elapsed time: 7845.927 ms
cuda_diffuse_and_price elapsed time: 7827.983 ms
cuda_diffuse_and_price elapsed time: 7821.746 ms
cuda_diffuse_and_price elapsed time: 7822.345 ms
cuda_diffuse_and_price elapsed time: 7821.404 ms
cuda_diffuse_and_price elapsed time: 7854.786 ms
cuda_diffuse_and_price elapsed time: 7859.27 ms
cuda_diffuse_and_price elapsed time: 7801.452 ms
cuda_diffuse_and_price elapsed time: 7790.668 ms
cuda_diffuse_and_price elapsed time: 7822.288 ms
cuda_diffuse_and_price elapsed time: 7809.116 ms
cuda_diffuse_and_price elapsed time: 7817.751 ms
cuda_diffuse_and_price elapsed time: 7865.257 ms
cuda_diffuse_and_price elapsed time: 7810.046 ms
cuda_diffuse_and_price elapsed time: 7793.457 ms
cuda_diffuse_and_price elapsed time: 7766.748 ms
cuda_diffuse_and_price elapsed time: 7867.743 ms
cuda_diffuse_and_price elapsed time: 7796.739 ms
cuda_diffuse_and_pric

In [31]:
bench_data = {}
bench_data['delta'] = bench_delta
bench_data['gamma'] = bench_gamma
bench_data['delta_sd'] = bench_delta_se
bench_timing = {'overhead':time_bench_overhead, 'rand':time_bench_rand, 'rein':(time_plus_rein, time_minus_rein), 'simul':time_bench_simul, 'sub':time_bench_sub}
bench_data['timing'] = bench_timing

In [32]:
with open('{}_{}_{}.pickle'.format(num_rates, num_spreads, seed_diff), 'wb') as file:
    pickle.dump((len(index_para), linear_data, smart_data, bench_data), file)